有两个神经网络，结构如下：

DNN： 784-2048-784，BNN： 784-512-10

将DNN的输出作为BNN的输入，串联成一个新的神经网络：784-2048-784-512-10

## DNN

In [42]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

dataset = "mnist"
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
# (X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# 归一化
# X_train = X_train / 255.
# X_test = X_test / 255.

dnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(2048, activation="relu"),
    tf.keras.layers.Dense(784, activation="relu"),
#     tf.keras.layers.Dense(4096, activation="relu"),
#     tf.keras.layers.Dense(256, activation="relu"),
#     tf.keras.layers.Dense(128, activation="relu"),
#     tf.keras.layers.Dense(64, activation="relu"),
#     tf.keras.layers.Dense(10, activation="softmax")
])
dnn_model.summary()

loss = tf.keras.losses.SparseCategoricalCrossentropy()
epochs = 1

dnn_model.compile(optimizer=tf.optimizers.Adam(), loss=loss, metrics=['accuracy'])
dnn_model.fit(X_train, y_train, epochs=epochs)

dnn_model_path = "model/dnn_model_" + dataset + "_epochs_" + str(epochs) + ".h5"
dnn_model.save(dnn_model_path)
print(dnn_model_path + "保存完成")

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 dense_9 (Dense)             (None, 2048)              1607680   
                                                                 
 dense_10 (Dense)            (None, 784)               1606416   
                                                                 
Total params: 3,214,096
Trainable params: 3,214,096
Non-trainable params: 0
_________________________________________________________________
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_conve

In [43]:
dnn_model = tf.keras.models.load_model(dnn_model_path)
dnn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 dense_9 (Dense)             (None, 2048)              1607680   
                                                                 
 dense_10 (Dense)            (None, 784)               1606416   
                                                                 
Total params: 3,214,096
Trainable params: 3,214,096
Non-trainable params: 0
_________________________________________________________________


In [44]:
bnn_X_train = dnn_model.predict(X_train)
bnn_X_test = dnn_model.predict(X_test)
bnn_X_train[0]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


## BNN

In [63]:
bnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(784,)),
    tf.keras.layers.Dense(512, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax"),
])
bnn_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_7 (Flatten)         (None, 784)               0         
                                                                 
 dense_15 (Dense)            (None, 512)               401920    
                                                                 
 dense_16 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [64]:
###
import sys, os
from pathlib import Path

path = Path(os.getcwd())
sys.path.append(str(path.parent))

import BayesKeras
import BayesKeras.optimizers as optimizers

arg = {'eps': 0.11, 'lam': 0.25, 'rob': 0, 'opt': 'BBB', 'gpu': '-1'}

eps = arg['eps']
lam = arg['lam']
rob = arg['rob']
optim = arg['opt']
gpu = arg['gpu']
os.environ['CUDA_VISIBLE_DEVICES'] = gpu

inf = 2
full_covar = False
if (optim == 'VOGN'):
    # was 0.25 for a bit
    inf = 2
    learning_rate = 0.35
    decay = 0.0
    opt = optimizers.VariationalOnlineGuassNewton()
elif (optim == 'BBB'):
    inf = 10
    learning_rate = 0.45
    decay = 0.0
    opt = optimizers.BayesByBackprop()
elif (optim == 'SWAG'):
    learning_rate = 0.01
    decay = 0.0
    opt = optimizers.StochasticWeightAveragingGaussian()
elif (optim == 'SWAG-FC'):
    learning_rate = 0.01
    decay = 0.0
    full_covar = True
    opt = optimizers.StochasticWeightAveragingGaussian()
elif (optim == 'SGD'):
    learning_rate = 1.0
    decay = 0.0
    opt = optimizers.StochasticGradientDescent()
elif (optim == 'NA'):
    inf = 2
    learning_rate = 0.001
    decay = 0.0
    opt = optimizers.NoisyAdam()
elif (optim == 'ADAM'):
    learning_rate = 0.00001
    decay = 0.0
    opt = optimizers.Adam()
elif (optim == 'HMC'):
    learning_rate = 0.075
    decay = 0.0
    inf = 250
    linear_schedule = False
    opt = optimizers.HamiltonianMonteCarlo()

# Compile the model to train with Bayesian inference
if (rob == 0):
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
elif (rob != 0):
    loss = BayesKeras.optimizers.losses.robust_crossentropy_loss



This optimizer does not have a default compilation method. Please make sure to call the correct .compile method before use.


### 编译和训练

In [65]:
bnn_model = opt.compile(bnn_model, loss_fn=loss, learning_rate=learning_rate, epochs=1,
                          batch_size=128, linear_schedule=True,
                          decay=decay, robust_train=rob, inflate_prior=inf,
                          burn_in=3, steps=25, b_steps=20, epsilon=eps, rob_lam=lam)  # , preload="SGD_FCN_Posterior_1")

BayesKeras: Using implicit prior
sha: (784, 512), std: 0.11293848786315641
sha: (512, 10), std: 0.13975424859373686
BayesKeras: Using implicit prior
sha: (784, 512), std: 0.11293848786315641
sha: (512, 10), std: 0.13975424859373686
计算后验方差
BayesKeras: Using passed loss_fn as the data likelihood in the KL loss


In [66]:
# steps was 50
# Train the model on your data
bnn_model.train(bnn_X_train, y_train, bnn_X_test, y_test)

100%|██████████████████████████████| 469/469 [01:46<00:00,  4.39it/s]


Epoch 1, loss: nan, acc: 0.099, val_loss: nan, val_acc: 0.098


In [67]:
# Save your approxiate Bayesian posterior
bnn_model.save("%s_FCN_Posterior_%s_dnn_bnn" % (optim, rob))

D:\Users\T470\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


### 预测

DNN的输出作为BNN的输入

In [69]:
pred = bnn_model.predict(bnn_X_test)
pred[0]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)>

In [70]:
pred[1]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)>

In [71]:
y_pred = np.argmax(pred, axis=1)
y_pred[:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)